## Importando Bibliotecas Necessárias

links:
https://github.com/binance/binance-spot-api-docs/blob/master/rest-api.md

https://python-binance.readthedocs.io/en/latest/constants.html

In [1]:
import pandas as pd
import numpy as np
from binance.client import Client
from sqlalchemy import create_engine
import psycopg2
import os

## Conexão com a API da Binance

In [2]:
api_key = os.getenv(key='BINANCE_API_KEY')
api_secret = os.getenv(key='BINANCE_SECRET_KEY')

client = Client(api_key, api_secret)

## Definindo função para extração dos dados

In [3]:
def BinanceDataFrame(symbol):
    # Consultando a API da Binance
    ar = np.array(
    client.get_historical_klines(
        symbol=symbol,
        interval=Client.KLINE_INTERVAL_1DAY,
        start_str='1 Jan, 2020'
        )
    )
    
    # Transformando dados em um DataFrame
    df = pd.DataFrame(
    ar, 
    dtype=float, 
    columns=(
        'Open_Time',
        'Open_Price',
        'High_Price',
        'Low_Price',
        'Close_Price',
        'Volume',
        'Close_Time',
        'Quote_Asset_Volume',
        'Number_of_Trades',
        'Taker buy base asset volume',
        'Taker buy quote asset volume',
        'Unused field'
    ))
    
    # Convertendo as colunas para datetime

    df['Open_Time'] = pd.to_datetime(df['Open_Time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
    df['Close_Time'] = pd.to_datetime(df['Close_Time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
    
    # Selecionando colunas desejadas
    df = df[[
        'Open_Time',
        'Open_Price',
        'High_Price',
        'Low_Price',
        'Close_Price',
        'Volume',
        'Close_Time',
        'Number_of_Trades'
    ]]

    return df

## Utilizando a função para trazer os dados das cryptos

In [4]:
btc = BinanceDataFrame('BTCUSDT')

eth = BinanceDataFrame('ETHUSDT')

## Criando variáveis (Média Móvel e Bandas de Bollinger)

In [5]:
def Bollinger(df):
    # Média móvel
    df['Media_Movel'] = df['Close_Price'].shift(1).rolling(20).mean()

    # Desvio padrão
    df['Desvio_Padrao'] = df['Close_Price'].shift(1).rolling(20).std()

    # Banda superior
    df['Banda_Superior'] = df['Media_Movel'] + 2*btc['Desvio_Padrao']

    # Banda inferior
    df['Banda_Inferior'] = df['Media_Movel'] - 2*btc['Desvio_Padrao']

    return df

In [6]:
btc = Bollinger(btc)

eth = Bollinger(eth)

In [7]:
btc.head(25)

,Open_Time,Open_Price,High_Price,Low_Price,Close_Price,Volume,Close_Time,Number_of_Trades,Media_Movel,Desvio_Padrao,Banda_Superior,Banda_Inferior
0,2019-12-31 21:00:00-03:00,7195.24,7255.00,7175.15,7200.85,16792.388165,2020-01-01 20:59:59.999000064-03:00,194010.0,NaN,NaN,NaN,NaN
1,2020-01-01 21:00:00-03:00,7200.77,7212.50,6924.74,6965.71,31951.483932,2020-01-02 20:59:59.999000064-03:00,302667.0,NaN,NaN,NaN,NaN
2,2020-01-02 21:00:00-03:00,6965.49,7405.00,6871.04,7344.96,68428.500451,2020-01-03 20:59:59.999000064-03:00,519854.0,NaN,NaN,NaN,NaN
3,2020-01-03 21:00:00-03:00,7345.00,7404.00,7272.21,7354.11,29987.974977,2020-01-04 20:59:59.999000064-03:00,279370.0,NaN,NaN,NaN,NaN
4,2020-01-04 21:00:00-03:00,7354.19,7495.00,7318.00,7358.75,38331.085604,2020-01-05 20:59:59.999000064-03:00,329209.0,NaN,NaN,NaN,NaN
5,2020-01-05 21:00:00-03:00,7357.64,7795.34,7346.76,7758.00,54635.695316,2020-01-06 20:59:59.999000064-03:00,449937.0,NaN,NaN,NaN,NaN
6,2020-01-06 21:00:00-03:00,7758.90,8207.68,7723.71,8145.28,91171.684661,2020-01-07 20:59:59.999000064-03:00,685315.0,NaN,NaN,NaN,NaN
7,2020-01-07 21:00:00-03:00,8145.92,8455.00,7870.00,8055.98,112622.642640,2020-01-08 20:59:59.999000064-03:00,830375.0,NaN,NaN,NaN,NaN
8,2020-01-08 21:00:00-03:00,8054.72,8055.96,7750.00,7817.76,64239.519830,2020-01-09 20:59:59.999000064-03:00,478800.0,NaN,NaN,NaN,NaN
9,2020-01-09 21:00:00-03:00,7817.74,8199.00,7672.00,8197.02,82406.777448,2020-01-10 20:59:59.999000064-03:00,606101.0,NaN,NaN,NaN,NaN


In [8]:
eth.head(2)

,Open_Time,Open_Price,High_Price,Low_Price,Close_Price,Volume,Close_Time,Number_of_Trades,Media_Movel,Desvio_Padrao,Banda_Superior,Banda_Inferior
0,2019-12-31 21:00:00-03:00,129.16,133.05,128.68,130.77,144770.52197,2020-01-01 20:59:59.999000064-03:00,75888.0,NaN,NaN,NaN,NaN
1,2020-01-01 21:00:00-03:00,130.72,130.78,126.38,127.19,213757.05806,2020-01-02 20:59:59.999000064-03:00,96193.0,NaN,NaN,NaN,NaN


## Conectando ao banco de dados PostgreSQL

In [57]:
# Fazendo a conexão

try:
    conn = psycopg2.connect(database = "postgres", user = "postgres", password = "feararo", host = "localhost", port = "5432")
except:
    print("I am unable to connect to the database") 

cur = conn.cursor()

# Criando a tabela
try:
    cur.execute("""
                    CREATE TABLE IF NOT EXISTS bitcoin (
                    Open_Time TIMESTAMP,
                    Open_Price FLOAT,
                    High_Price FLOAT,
                    Low_Price FLOAT,
                    Close_Price FLOAT,
                    Volume INT,
                    Close_Time TIMESTAMP,
                    Number_of_Trades INT,
                    Media_Movel VARCHAR,
                    Desvio_Padrao VARCHAR,
                    Banda_Superior VARCHAR,
                    Banda_Inferior VARCHAR);
                """)
except:
    print("I can't drop our test database!")

conn.commit()
conn.close()
cur.close()

## Inserindo os dados na base

In [65]:
# Fazendo a conexão

try:
    conn = psycopg2.connect(database = "postgres", user = "postgres", password = "feararo", host = "localhost", port = "5432")
except:
    print("I am unable to connect to the database") 

cur = conn.cursor()

# Inserindo os dados

try:
    for i in btc.index:
        cur.execute(f"""
                    INSERT INTO bitcoin (Open_Time, 
                                        Open_Price, 
                                        High_Price, 
                                        Low_Price, 
                                        Close_Price, 
                                        Volume, 
                                        Close_Time, 
                                        Number_of_Trades, 
                                        Media_Movel, 
                                        Desvio_Padrao, 
                                        Banda_Superior, 
                                        Banda_Inferior)
                    VALUES ('{btc['Open_Time'][i]}',
                            {btc['Open_Price'][i]},
                            {btc['High_Price'][i]},
                            {btc['Low_Price'][i]},
                            {btc['Close_Price'][i]},
                            {btc['Volume'][i]},
                            '{btc['Close_Time'][i]}',
                            {btc['Number_of_Trades'][i]},
                            '{btc['Media_Movel'][i]}',
                            '{btc['Desvio_Padrao'][i]}',
                            '{btc['Banda_Superior'][i]}',
                            '{btc['Banda_Inferior'][i]}',
                            )""")
except:
    print("I am unable to insert the data")

I am unable to insert the data


In [61]:
queries = list()

for i in btc.index:
    query = (f"""
                INSERT INTO bitcoin (Open_Time,
                                    Open_Price, 
                                    High_Price, 
                                    Low_Price, 
                                    Close_Price, 
                                    Volume, 
                                    Close_Time, 
                                    Number_of_Trades, 
                                    Media_Movel, 
                                    Desvio_Padrao, 
                                    Banda_Superior, 
                                    Banda_Inferior)
                VALUES ('{btc['Open_Time'][i]}',
                        {btc['Open_Price'][i]},
                        {btc['High_Price'][i]},
                        {btc['Low_Price'][i]},
                        {btc['Close_Price'][i]},
                        {btc['Volume'][i]},
                        '{btc['Close_Time'][i]}',
                        {btc['Number_of_Trades'][i]},
                        '{btc['Media_Movel'][i]}',
                        '{btc['Desvio_Padrao'][i]}',
                        '{btc['Banda_Superior'][i]}',
                        '{btc['Banda_Inferior'][i]}')""")
    queries.append(' '.join(query.split()))